<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">

*このノートブックには、Jake VanderPlas による [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) からの抜粋が含まれています。コンテンツは利用可能です [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do).*

※テキストは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で、コードは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で公開しています。このコンテンツが役立つと思われる場合は、[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode) による作業のサポートを検討してください!*

<!--ナビゲーション-->
< [Hierarchical Indexing](03.05-Hierarchical-Indexing.ipynb) | [Hierarchical Indexing](03.05-Hierarchical-Indexing.ipynb) | [Hierarchical Indexing](03.05-Hierarchical-Indexing.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/03.06-Concat-And-Append.ipynb"><img align="left" src=" https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>


# データセットの結合: 連結と追加

データに関する最も興味深い研究のいくつかは、さまざまなデータ ソースを組み合わせることから得られます。
これらの操作には、2 つの異なるデータセットの非常に単純な連結から、データセット間のオーバーラップを正しく処理するより複雑なデータベース スタイルの結合およびマージまで、あらゆる操作が含まれます。
``Series`` と ``DataFrame`` は、この種の操作を念頭に置いて構築されており、Pandas には、この種のデータ ラングリングを高速かつ簡単にする関数とメソッドが含まれています。

ここでは、 pd.concat 関数を使用した Series と DataFrame の単純な連結を見ていきます。後で、Pandas に実装されたより洗練されたメモリ内マージと結合について詳しく説明します。

標準のインポートから始めます。

In [1]:
import pandas as pd
import numpy as np

便宜上、以下で役立つ特定の形式の ``DataFrame`` を作成するこの関数を定義します。

In [2]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


さらに、複数の ``DataFrame`` を並べて表示できるようにするクイック クラスを作成します。このコードは、IPython がリッチ オブジェクト表示を実装するために使用する特別な ``_repr_html_`` メソッドを利用します。

In [3]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

これの使用法は、次のセクションで説明を続けるにつれて、より明確になります。

## 思い出してください: NumPy 配列の連結

Series オブジェクトと DataFrame オブジェクトの連結は、Numpy 配列の連結に非常に似ています。これは、[The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb) で説明されているように、np.concatenate 関数を介して行うことができます。
これを使用すると、2 つ以上の配列の内容を 1 つの配列に結合できることを思い出してください。

In [4]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

最初の引数は、連結する配列のリストまたはタプルです。
さらに、結果が連結される軸を指定できる ``axis`` キーワードを取ります:

In [5]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

## ``pd.concat`` による単純な連結

Pandas には関数 pd.concat() があり、これは np.concatenate と同様の構文を持ちますが、ここで説明するいくつかのオプションが含まれています。

```python
# Signature in Pandas v0.18
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
```

``pd.concat()`` は、``np.concatenate()`` が配列の単純な連結に使用できるように、``Series`` または ``DataFrame`` オブジェクトの単純な連結に使用できます。

In [6]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

``DataFrame`` などの高次元オブジェクトを連結するためにも機能します。

In [7]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2])')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


デフォルトでは、連結は ``DataFrame`` 内で行単位で行われます (つまり、 ``axis=0``)。
``np.concatenate`` と同様に、``pd.concat`` では連結が行われる軸を指定できます。
次の例を検討してください。

In [8]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis='col')")

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4], axis='col')
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1

同様に ``axis=1`` を指定することもできました。ここでは、より直感的な ``axis='col'`` を使用しました。

### インデックスの重複

``np.concatenate`` と ``pd.concat`` の重要な違いの 1 つは、結果に重複したインデックスが含まれていても、Pandas 連結は *インデックスを保持する* ことです!
次の簡単な例を考えてみましょう。

In [9]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # make duplicate indices!
display('x', 'y', 'pd.concat([x, y])')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


結果でインデックスが繰り返されていることに注意してください。
これは ``DataFrame`` 内では有効ですが、多くの場合、望ましくない結果になります。
``pd.concat()`` はそれを処理するいくつかの方法を提供します。

#### 繰り返しをエラーとして捉える

pd.concat() の結果のインデックスが重複していないことを単純に確認したい場合は、verify_integrity フラグを指定できます。
これを True に設定すると、重複するインデックスがある場合、連結によって例外が発生します。
わかりやすくするために、エラーメッセージをキャッチして出力する例を次に示します。

In [10]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: [0, 1]


#### インデックスを無視する

インデックス自体は問題にならない場合があり、単純に無視したい場合があります。
このオプションは ``ignore_index`` フラグを使用して指定できます。
これを true に設定すると、連結によって結果の ``Series`` の新しい整数インデックスが作成されます。

In [11]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


#### MultiIndex キーの追加

もう 1 つのオプションは、データ ソースのラベルを指定するために ``keys`` オプションを使用することです。結果は、データを含む階層的に索引付けされたシリーズになります。

In [12]:
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

x
    A   B
0  A0  B0
1  A1  B1

y
    A   B
0  A2  B2
1  A3  B3

pd.concat([x, y], keys=['x', 'y'])
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

その結果、複数のインデックスが付けられた ``DataFrame`` が作成され、[Hierarchical Indexing](03.05-Hierarchical-Indexing.ipynb) で説明されているツールを使用して、このデータを目的の表現に変換できます。

### 結合による連結

先ほど見た単純な例では、主に共有列名を持つ ``DataFrame`` を連結していました。
実際には、異なるソースからのデータは異なる列名のセットを持っている可能性があり、この場合には ``pd.concat`` がいくつかのオプションを提供します。
次の 2 つの ``DataFrame`` の連結を考えてみましょう。これらにはいくつかの (すべてではない!) 列が共通しています:

In [13]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6])')

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

デフォルトでは、データが利用できないエントリには NA 値が入力されます。
これを変更するには、concatenate 関数の ``join`` および ``join_axes`` パラメータにいくつかのオプションの 1 つを指定できます。
デフォルトでは、結合は入力列の結合 (``join='outer'``) ですが、``join='inner'`` を使用してこれを列の共通部分に変更できます:

In [14]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')")

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join='inner')
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4

別のオプションは、インデックス オブジェクトのリストを取る ``join_axes`` 引数を使用して、残りの列のインデックスを直接指定することです。
ここでは、返される列が最初の入力のものと同じであることを指定します。

In [15]:
display('df5', 'df6',
        "pd.concat([df5, df6], join_axes=[df5.columns])")

,A,B,C
1,A1,B1,C1
2,A2,B2,C2
,B,C,D
3,B3,C3,D3
4,B4,C4,D4
,A,B,C
1,A1,B1,C1
2,A2,B2,C2
3,NaN,B3,C3
4,NaN,B4,C4


``pd.concat`` 関数のオプションを組み合わせることで、2 つのデータセットを結合する際にさまざまな動作が可能になります。これらのツールを独自のデータに使用するときは、これらの点に留意してください。

### ``append()`` メソッド

配列の直接連結は非常に一般的であるため、Series オブジェクトと DataFrame オブジェクトには、より少ないキーストロークで同じことを実行できる append メソッドがあります。
たとえば、pd.concat([df1, df2])) を呼び出す代わりに、単に df1.append(df2) を呼び出すことができます。

In [16]:
display('df1', 'df2', 'df1.append(df2)')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


Python リストの ``append()`` および ``extend()`` メソッドとは異なり、Pandas の ``append()`` メソッドは元のオブジェクトを変更しないことに注意してください。代わりに、新しいオブジェクトを作成します。組み合わせたデータで。
また、新しいインデックス * および * データ バッファーの作成を伴うため、あまり効率的な方法ではありません。
したがって、複数の ``append`` 操作を実行する予定がある場合は、一般に ``DataFrame`` のリストを作成し、それらすべてを一度に ``concat()`` 関数に渡す方がよいでしょう。

次のセクションでは、複数のソースからのデータを結合するための別のより強力なアプローチ、 pd.merge で実装されたデータベース スタイルのマージ/結合について見ていきます。
``concat()``、``append()``、および関連する機能の詳細については、Pandas ドキュメントの ["Merge, Join, and Concatenate" section](http://pandas.pydata.org/pandas-docs/stable/merging.html) を参照してください。

<!--ナビゲーション-->
< [Hierarchical Indexing](03.05-Hierarchical-Indexing.ipynb) | [Hierarchical Indexing](03.05-Hierarchical-Indexing.ipynb) | [Hierarchical Indexing](03.05-Hierarchical-Indexing.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/03.06-Concat-And-Append.ipynb"><img align="left" src=" https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>
